# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

df.columns = df.columns.str.lower().str.replace(' ', '_')
df['total_claim_amount'] = pd.to_numeric(df['total_claim_amount'], errors='coerce')
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
df['monthly_premium_auto'] = pd.to_numeric(df['monthly_premium_auto'], errors='coerce')

filtered_df = df[(df['total_claim_amount'] < 1000) & (df['response'] == 'Yes')]

yes_df = df[df['response'] == 'Yes']
grouped = yes_df.groupby(['policy_type', 'gender']).agg({
    'monthly_premium_auto': 'mean',
    'customer_lifetime_value': 'mean',
    'total_claim_amount': 'mean'
}).reset_index()

states_counts = df['state'].value_counts()
states_over_500 = states_counts[states_counts > 500]

clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()

df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
df['month'] = df['effective_to_date'].dt.month_name()
df['month'] = pd.Categorical(df['month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December'], ordered=True)

policies_by_state_month = df.groupby(['state', 'month']).size().unstack(fill_value=0)

top_states = df['state'].value_counts().nlargest(3).index
top_states_df = df[df['state'].isin(top_states)]
top_policies_by_month = top_states_df.groupby(['state', 'month']).size().unstack(fill_value=0)

response_rate = df.groupby('sales_channel')['response'].value_counts(normalize=True).unstack().fillna(0)
response_rate = response_rate[['Yes']].rename(columns={'Yes': 'response_rate'})

print(filtered_df.head())
print(grouped)
print(states_over_500)
print(clv_stats)
print(policies_by_state_month)
print(top_policies_by_month)
print(response_rate)


    unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11         Employed      F  ...   

    number_of_open_complaints number_of_policies     policy_type  \
3                         0.0                  2  Corp

/var/folders/9p/zf59q68s4fl48rcv9jgn0xcm0000gn/T/ipykernel_60820/4146974252.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
/var/folders/9p/zf59q68s4fl48rcv9jgn0xcm0000gn/T/ipykernel_60820/4146974252.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  policies_by_state_month = df.groupby(['state', 'month']).size().unstack(fill_value=0)
/var/folders/9p/zf59q68s4fl48rcv9jgn0xcm0000gn/T/ipykernel_60820/4146974252.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observe